### We start by importing our libraries 


In [1]:
!pip install BeautifulSoup4
!pip install requests


In [2]:
import numpy as np 
import pandas as pd 
from pandas.io.json import json_normalize
import json
import requests
from bs4 import BeautifulSoup 
from sklearn.cluster import KMeans 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium


We then import the data (this data is from week 3) , join it and changed it to a data frame

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

text_result = requests.get(url).text 
html_parsed_result = BeautifulSoup(text_result, 'html.parser')

Neighborhood_info_table = html_parsed_result.find('table', class_ = 'wikitable')
Neighborhood_rows = Neighborhood_info_table.find_all('tr')

Neighborhood_info = []
for row in Neighborhood_rows:
    info = row.text.split('\n')[1:-1] 
    Neighborhood_info.append(info)
Neighborhood_df = pd.DataFrame(Neighborhood_info[1:], columns=Neighborhood_info[0])
not_assigned_boroughs = Neighborhood_df.index[Neighborhood_df['Borough'] == 'Not assigned']
not_assigned_neighbourhoods = Neighborhood_df.index[Neighborhood_df['Neighbourhood'] == 'Not assigned']
not_assigned_neighbourhoods_and_boroughs = not_assigned_boroughs & not_assigned_neighbourhoods
Neighborhood_df.drop(Neighborhood_df.index[not_assigned_boroughs], inplace=True)
Neighborhood_df.reset_index(drop=True, inplace=True)
not_assigned_neighbourhoods = Neighborhood_df.index[Neighborhood_df['Neighbourhood'] == 'Not assigned'] 
for idx in not_assigned_neighbourhoods:
    Neighborhood_df['Neighbourhood'][idx] = Neighborhood_df['Borough'][idx]
Neighborhood_df = Neighborhood_df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: ", ".join(x.astype(str))).reset_index()
Neighborhood_df = Neighborhood_df.sample(frac=1).reset_index(drop=True)

In [4]:
url2="http://cocl.us/Geospatial_data"
geo_data=pd.read_csv(url2)
geo_data.rename(columns={'Postal Code':'Postcode'},inplace=True)
df_toronto=pd.merge(Neighborhood_df,geo_data,on='Postcode')
df_toronto.head(20)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
1,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
2,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So...",43.636258,-79.498509
3,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
4,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
5,M4C,East York,Woodbine Heights,43.695344,-79.318389
6,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
7,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678
8,M4E,East Toronto,The Beaches,43.676357,-79.293031
9,M4J,East York,East Toronto,43.685347,-79.338106


We now map the city of Toronto with all its postal codes regions using the latitude and longtitude given by google and our imported folium map

In [5]:
toronto_latitude = 43.6932; toronto_longitude = -79.3832
map_toronto = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start = 10.7)
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    

map_toronto

We focus now on the borough of Scarborough 

In [6]:
scarborough_data = df_toronto[df_toronto['Borough'] == 'Scarborough']
scarborough_data = scarborough_data.reset_index(drop=True)
scarborough_data.head(20)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
3,M1R,Scarborough,"Maryvale, Wexford",43.750072,-79.295849
4,M1W,Scarborough,L'Amoreaux West,43.799525,-79.318389
5,M1T,Scarborough,"Clarks Corners, Sullivan, Tam O'Shanter",43.781638,-79.304302
6,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
7,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
8,M1G,Scarborough,Woburn,43.770992,-79.216917
9,M1V,Scarborough,"Agincourt North, L'Amoreaux East, Milliken, St...",43.815252,-79.284577


Let's create a map of Scarborough and all it's neighborhoods

In [7]:
address_scar = 'Scarborough, Toronto'
latitude_scar = 43.773077
longitude_scar = -79.257774
map_Scarborough = folium.Map(location=[latitude_scar, longitude_scar], zoom_start=11.5)

for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius = 10,
        popup = label,
        color ='blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7).add_to(map_Scarborough)  
    
map_Scarborough

We now use foursqaure data base to obtain data on the venues of each neighborhood

In [8]:
# @hidden_cell
CLIENT_ID = 'MITOL0ANQWPEHHS4FAEWNASU1EG4N2LFWVFCMTWQNYS0211P' 
CLIENT_SECRET = 'H3PBT4F1ZISCILN3AX0CFQ1WMEN043PJIRPPQXDA3AARBRKG' 
VERSION = '20181102'

In [9]:
def foursquare_crawler (postal_code_list, neighborhood_list, lat_list, lng_list, LIMIT = 500, radius = 1000):
    result_ds = []
    counter = 0
    for postal_code, neighborhood, lat, lng in zip(postal_code_list, neighborhood_list, lat_list, lng_list):
         
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, 
            lat, lng, radius, LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        tmp_dict = {}
        tmp_dict['Postal Code'] = postal_code; tmp_dict['Neighborhood(s)'] = neighborhood; 
        tmp_dict['Latitude'] = lat; tmp_dict['Longitude'] = lng;
        tmp_dict['Crawling_result'] = results;
        result_ds.append(tmp_dict)
        counter += 1
        print('{}.'.format(counter))
        print('Data is Obtained, for the Postal Code {} (and Neighborhoods {}) SUCCESSFULLY.'.format(postal_code, neighborhood))
    return result_ds;
print('Crawling different neighborhoods inside "Scarborough"')
Scarborough_foursquare_dataset = foursquare_crawler(list(scarborough_data['Postcode']),
                                                   list(scarborough_data['Neighbourhood']),
                                                   list(scarborough_data['Latitude']),
                                                   list(scarborough_data['Longitude']),)

Crawling different neighborhoods inside "Scarborough"
1.
Data is Obtained, for the Postal Code M1B (and Neighborhoods Rouge, Malvern) SUCCESSFULLY.
2.
Data is Obtained, for the Postal Code M1C (and Neighborhoods Highland Creek, Rouge Hill, Port Union) SUCCESSFULLY.
3.
Data is Obtained, for the Postal Code M1M (and Neighborhoods Cliffcrest, Cliffside, Scarborough Village West) SUCCESSFULLY.
4.
Data is Obtained, for the Postal Code M1R (and Neighborhoods Maryvale, Wexford) SUCCESSFULLY.
5.
Data is Obtained, for the Postal Code M1W (and Neighborhoods L'Amoreaux West) SUCCESSFULLY.
6.
Data is Obtained, for the Postal Code M1T (and Neighborhoods Clarks Corners, Sullivan, Tam O'Shanter) SUCCESSFULLY.
7.
Data is Obtained, for the Postal Code M1H (and Neighborhoods Cedarbrae) SUCCESSFULLY.
8.
Data is Obtained, for the Postal Code M1N (and Neighborhoods Birch Cliff, Cliffside West) SUCCESSFULLY.
9.
Data is Obtained, for the Postal Code M1G (and Neighborhoods Woburn) SUCCESSFULLY.
10.
Data is Ob

We clean the data received from Foursquare Database

In [10]:

def get_venue_dataset(foursquare_dataset):
    result_df = pd.DataFrame(columns = ['Postal Code', 'Neighborhood', 
                                           'Neighborhood Latitude', 'Neighborhood Longitude',
                                          'Venue', 'Venue Summary', 'Venue Category', 'Distance'])
    
    for neigh_dict in foursquare_dataset:
        postal_code = neigh_dict['Postal Code']; neigh = neigh_dict['Neighborhood(s)']
        lat = neigh_dict['Latitude']; lng = neigh_dict['Longitude']
        print('Number of Venuse in Coordination "{}" Posal Code and "{}" Negihborhood(s) is:'.format(postal_code, neigh))
        print(len(neigh_dict['Crawling_result']))
        
        for venue_dict in neigh_dict['Crawling_result']:
            summary = venue_dict['reasons']['items'][0]['summary']
            name = venue_dict['venue']['name']
            dist = venue_dict['venue']['location']['distance']
            cat =  venue_dict['venue']['categories'][0]['name']
     
            result_df = result_df.append({'Postal Code': postal_code, 'Neighborhood': neigh, 
                              'Neighborhood Latitude': lat, 'Neighborhood Longitude':lng,
                              'Venue': name, 'Venue Summary': summary, 
                              'Venue Category': cat, 'Distance': dist}, ignore_index = True)
    return(result_df)

In [11]:
scarborough_venues = get_venue_dataset(Scarborough_foursquare_dataset)

Number of Venuse in Coordination "M1B" Posal Code and "Rouge, Malvern" Negihborhood(s) is:
18
Number of Venuse in Coordination "M1C" Posal Code and "Highland Creek, Rouge Hill, Port Union" Negihborhood(s) is:
5
Number of Venuse in Coordination "M1M" Posal Code and "Cliffcrest, Cliffside, Scarborough Village West" Negihborhood(s) is:
13
Number of Venuse in Coordination "M1R" Posal Code and "Maryvale, Wexford" Negihborhood(s) is:
28
Number of Venuse in Coordination "M1W" Posal Code and "L'Amoreaux West" Negihborhood(s) is:
24
Number of Venuse in Coordination "M1T" Posal Code and "Clarks Corners, Sullivan, Tam O'Shanter" Negihborhood(s) is:
32
Number of Venuse in Coordination "M1H" Posal Code and "Cedarbrae" Negihborhood(s) is:
28
Number of Venuse in Coordination "M1N" Posal Code and "Birch Cliff, Cliffside West" Negihborhood(s) is:
15
Number of Venuse in Coordination "M1G" Posal Code and "Woburn" Negihborhood(s) is:
8
Number of Venuse in Coordination "M1V" Posal Code and "Agincourt North

We explore all of the venues in each neighborhood

In [12]:
scarborough_venues.head()

,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Venue Category,Distance
0,M1B,"Rouge, Malvern",43.806686,-79.194353,Images Salon & Spa,This spot is popular,Spa,595
1,M1B,"Rouge, Malvern",43.806686,-79.194353,Caribbean Wave,This spot is popular,Caribbean Restaurant,912
2,M1B,"Rouge, Malvern",43.806686,-79.194353,Staples Morningside,This spot is popular,Paper / Office Supplies Store,735
3,M1B,"Rouge, Malvern",43.806686,-79.194353,Wendy's,This spot is popular,Fast Food Restaurant,600
4,M1B,"Rouge, Malvern",43.806686,-79.194353,Harvey's,This spot is popular,Fast Food Restaurant,796


In [13]:
scarborough_venues.to_csv('scarborough_venues.csv')
scarborough_venues = pd.read_csv('scarborough_venues.csv')
neighborhood_list = list(scarborough_venues['Neighborhood'].unique())
neigh_venue_summary = scarborough_venues.groupby('Neighborhood').count()
neigh_venue_summary.drop(columns = ['Unnamed: 0']).head()



,Postal Code,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Venue Category,Distance
Neighborhood,,,,,,,
Agincourt,51,51,51,51,51,51,51
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",27,27,27,27,27,27,27
"Birch Cliff, Cliffside West",15,15,15,15,15,15,15
Cedarbrae,28,28,28,28,28,28,28
"Clairlea, Golden Mile, Oakridge",29,29,29,29,29,29,29


In [14]:
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

print('Here is the list of different categories:')
list(scarborough_venues['Venue Category'].unique())

There are 113 uniques categories.
Here is the list of different categories:


['Spa',
 'Caribbean Restaurant',
 'Paper / Office Supplies Store',
 'Fast Food Restaurant',
 'Coffee Shop',
 'Hobby Shop',
 'Construction & Landscaping',
 'African Restaurant',
 'Chinese Restaurant',
 'Greek Restaurant',
 'Fruit & Vegetable Store',
 'Gym',
 'Bakery',
 'Arts & Crafts Store',
 'Sandwich Place',
 'Park',
 'Burger Joint',
 'Italian Restaurant',
 'Breakfast Spot',
 'Playground',
 'Beach',
 'Furniture / Home Store',
 'Sports Bar',
 'Pizza Place',
 'Wings Joint',
 'Cajun / Creole Restaurant',
 'Vietnamese Restaurant',
 'Fish Market',
 'Korean Restaurant',
 'Middle Eastern Restaurant',
 'Seafood Restaurant',
 'Supermarket',
 'Grocery Store',
 'Indian Restaurant',
 'Restaurant',
 'Badminton Court',
 'Smoke Shop',
 'Bar',
 'Intersection',
 'Gas Station',
 'Moving Target',
 'Flea Market',
 'Soccer Field',
 'Hotpot Restaurant',
 'Pharmacy',
 'Japanese Restaurant',
 'Bubble Tea Shop',
 'Camera Store',
 'Bank',
 'Other Great Outdoors',
 'Gym Pool',
 'Noodle House',
 'Mexican Restaur

In [15]:
scarborough_onehot = pd.get_dummies(data = scarborough_venues, drop_first  = False, 
                              prefix = "", prefix_sep = "", columns = ['Venue Category'])
scarborough_onehot.head()

,Unnamed: 0,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Distance,African Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Badminton Court,Bakery,Bank,Bar,Beach,Beer Store,Bowling Alley,Breakfast Spot,Bubble Tea Shop,Burger Joint,Bus Line,Bus Station,Bus Stop,Café,Cajun / Creole Restaurant,Camera Store,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Construction & Landscaping,Convenience Store,Department Store,Dessert Shop,Diner,Discount Store,Electronics Store,Event Space,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Food & Drink Shop,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gas Station,General Entertainment,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Hobby Shop,Hong Kong Restaurant,Hotpot Restaurant,IT Services,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motorcycle Shop,Moving Target,Music Store,Noodle House,Optical Shop,Other Great Outdoors,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Playground,Pool,Pool Hall,Print Shop,Pub,Rental Car Location,Restaurant,Sandwich Place,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Soccer Field,Spa,Sports Bar,Sri Lankan Restaurant,Supermarket,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Thrift / Vintage Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wings Joint,Yoga Studio
0,0,M1B,"Rouge, Malvern",43.806686,-79.194353,Images Salon & Spa,This spot is popular,595,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,M1B,"Rouge, Malvern",43.806686,-79.194353,Caribbean Wave,This spot is popular,912,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,M1B,"Rouge, Malvern",43.806686,-79.194353,Staples Morningside,This spot is popular,735,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,M1B,"Rouge, Malvern",43.806686,-79.194353,Wendy's,This spot is popular,600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,M1B,"Rouge, Malvern",43.806686,-79.194353,Harvey's,This spot is popular,796,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [18]:
important_list_of_features = [
 
 'Neighborhood',
 'Neighborhood Latitude',
 'Neighborhood Longitude',
 'African Restaurant',
 'Asian Restaurant',
 'BBQ Joint',
 'Bakery',
 'Breakfast Spot',
 'Burger Joint',
 'Cajun / Creole Restaurant',
 'Cantonese Restaurant',
 'Caribbean Restaurant',
 'Chinese Restaurant',
 'Diner',
 'Fast Food Restaurant',
 'Fish Market',
 'Food & Drink Shop',
 'Fried Chicken Joint',
 'Fruit & Vegetable Store',
 'Greek Restaurant',
 'Grocery Store',
 'Hakka Restaurant',
 'Hotpot Restaurant',
 'Indian Restaurant',
 'Italian Restaurant',
 'Japanese Restaurant',
 'Korean Restaurant',
 'Latin American Restaurant',
 'Malay Restaurant',
 'Mediterranean Restaurant',
 'Mexican Restaurant',
 'Middle Eastern Restaurant',
 'Noodle House',
 'Pizza Place',
 'Restaurant',
 'Sandwich Place',
 'Seafood Restaurant',
 'Shanghai Restaurant',
 'Sushi Restaurant',
 'Taiwanese Restaurant',
 'Thai Restaurant',
 'Vegetarian / Vegan Restaurant',
 'Vietnamese Restaurant',
 'Wings Joint']

In [19]:
scarborough_onehot = scarborough_onehot[important_list_of_features].drop(
    columns = ['Neighborhood Latitude', 'Neighborhood Longitude']).groupby(
    'Neighborhood').sum()

scarborough_onehot.head(10)

,African Restaurant,Asian Restaurant,BBQ Joint,Bakery,Breakfast Spot,Burger Joint,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Diner,Fast Food Restaurant,Fish Market,Food & Drink Shop,Fried Chicken Joint,Fruit & Vegetable Store,Greek Restaurant,Grocery Store,Hakka Restaurant,Hotpot Restaurant,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Noodle House,Pizza Place,Restaurant,Sandwich Place,Seafood Restaurant,Shanghai Restaurant,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint
Neighborhood,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Agincourt,0,1,1,2,1,0,0,1,2,7,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,1,2,2,1,1,1,1,0,0,0,1,0
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",0,0,0,2,0,0,0,0,1,6,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,1,0,0,0,1,2,0,0,0,0,0,0,0,1,0,0
"Birch Cliff, Cliffside West",0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
Cedarbrae,0,1,0,4,0,1,0,0,1,1,0,1,0,0,1,0,0,1,1,0,2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1
"Clairlea, Golden Mile, Oakridge",0,0,0,2,0,0,0,0,0,0,1,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,1,1,0,0,0,0,0,0,0,0
"Clarks Corners, Sullivan, Tam O'Shanter",0,0,0,0,0,0,0,1,1,1,0,2,0,0,1,0,1,1,0,0,0,1,0,0,0,0,0,1,0,1,2,0,2,1,0,0,1,1,0,1,0
"Cliffcrest, Cliffside, Scarborough Village West",0,0,0,0,0,1,1,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,1
"Dorset Park, Scarborough Town Centre, Wexford Heights",0,2,0,2,0,2,0,0,0,2,0,2,0,0,1,0,0,1,0,0,2,1,0,0,1,0,0,0,0,0,1,1,1,0,0,0,0,0,0,1,0
"East Birchmount Park, Ionview, Kennedy Park",0,1,0,0,0,1,0,0,0,2,0,2,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0


We now integrate all different joints and restaurant

In [20]:
feat_name_list = list(scarborough_onehot.columns)
restaurant_list = []


for counter, value in enumerate(feat_name_list):
    if value.find('Restaurant') != (-1):
        restaurant_list.append(value)
        
scarborough_onehot['Total Restaurants'] = scarborough_onehot[restaurant_list].sum(axis = 1)
scarborough_onehot = scarborough_onehot.drop(columns = restaurant_list)


feat_name_list = list(scarborough_onehot.columns)
joint_list = []


for counter, value in enumerate(feat_name_list):
    if value.find('Joint') != (-1):
        joint_list.append(value)
        
scarborough_onehot['Total Joints'] = scarborough_onehot[joint_list].sum(axis = 1)
scarborough_onehot = scarborough_onehot.drop(columns = joint_list)
scarborough_onehot

,Bakery,Breakfast Spot,Diner,Fish Market,Food & Drink Shop,Fruit & Vegetable Store,Grocery Store,Noodle House,Pizza Place,Sandwich Place,Total Restaurants,Total Joints
Neighborhood,,,,,,,,,,,,
Agincourt,2,1,0,0,0,0,0,1,2,1,20,1
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,0,0,0,0,0,0,1,2,0,12,0
"Birch Cliff, Cliffside West",0,0,1,0,0,0,0,0,0,0,4,0
Cedarbrae,4,0,0,0,0,0,1,0,1,0,8,3
"Clairlea, Golden Mile, Oakridge",2,0,1,0,0,0,1,0,1,1,4,0
"Clarks Corners, Sullivan, Tam O'Shanter",0,0,0,0,0,0,1,1,2,2,12,1
"Cliffcrest, Cliffside, Scarborough Village West",0,0,0,0,0,0,0,0,3,0,3,2
"Dorset Park, Scarborough Town Centre, Wexford Heights",2,0,0,0,0,0,1,0,1,1,12,3
"East Birchmount Park, Ionview, Kennedy Park",0,0,0,0,0,0,2,0,1,1,5,1


We now run k-means to cluster the neighborhoods using k=5

In [21]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters = 5, random_state = 0).fit(scarborough_onehot)

In [22]:
means_df = pd.DataFrame(kmeans.cluster_centers_)
means_df.columns = scarborough_onehot.columns
means_df.index = ['G1','G2','G3','G4','G5']
means_df['Total Sum'] = means_df.sum(axis = 1)
means_df.sort_values(axis = 0, by = ['Total Sum'], ascending=False)

,Bakery,Breakfast Spot,Diner,Fish Market,Food & Drink Shop,Fruit & Vegetable Store,Grocery Store,Noodle House,Pizza Place,Sandwich Place,Total Restaurants,Total Joints,Total Sum
G3,2.000000,1.000000,0.000000,0.00,0.0,0.00,0.000000,1.000000,2.000000,1.0,20.000000,1.000000,28.000000
G1,1.333333,0.000000,0.000000,0.00,0.0,0.00,0.666667,0.666667,1.666667,1.0,12.000000,1.333333,18.666667
G4,1.750000,0.500000,0.000000,0.25,0.0,0.25,1.000000,0.000000,1.000000,0.5,7.750000,1.250000,14.250000
G5,0.000000,0.000000,0.000000,0.00,0.5,0.00,0.000000,0.000000,4.000000,0.5,3.500000,2.000000,10.500000
G2,0.333333,0.166667,0.333333,0.00,0.0,0.00,0.500000,0.000000,0.500000,0.5,3.666667,0.333333,6.333333


So G1 one is the best group

We then find the coressponding group to each neighborhood

In [23]:
neigh_summary = pd.DataFrame([scarborough_onehot.index, 1 + kmeans.labels_]).T
neigh_summary.columns = ['Neighborhood', 'Group']
neigh_summary

,Neighborhood,Group
0,Agincourt,3
1,"Agincourt North, L'Amoreaux East, Milliken, St...",1
2,"Birch Cliff, Cliffside West",2
3,Cedarbrae,4
4,"Clairlea, Golden Mile, Oakridge",2
5,"Clarks Corners, Sullivan, Tam O'Shanter",1
6,"Cliffcrest, Cliffside, Scarborough Village West",5
7,"Dorset Park, Scarborough Town Centre, Wexford ...",1
8,"East Birchmount Park, Ionview, Kennedy Park",2
9,"Guildwood, Morningside, West Hill",5


So the best neighborhoods are :

In [ ]:
neigh_summary[neigh_summary['Group'] == 3]

In [24]:
name_of_neigh = list(neigh_summary[neigh_summary['Group'] == 3]['Neighborhood'])[0]
scarborough_venues[scarborough_venues['Neighborhood'] == name_of_neigh].iloc[0,1:5].to_dict()

{'Postal Code': 'M1S',
 'Neighborhood': 'Agincourt',
 'Neighborhood Latitude': 43.7942003,
 'Neighborhood Longitude': -79.26202940000002}

Second Best :

In [25]:
neigh_summary[neigh_summary['Group'] == 1]

,Neighborhood,Group
1,"Agincourt North, L'Amoreaux East, Milliken, St...",1
5,"Clarks Corners, Sullivan, Tam O'Shanter",1
7,"Dorset Park, Scarborough Town Centre, Wexford ...",1


Third best:

In [26]:
neigh_summary[neigh_summary['Group'] == 2]

,Neighborhood,Group
2,"Birch Cliff, Cliffside West",2
4,"Clairlea, Golden Mile, Oakridge",2
8,"East Birchmount Park, Ionview, Kennedy Park",2
10,"Highland Creek, Rouge Hill, Port Union",2
14,Scarborough Village,2
15,Woburn,2


### The End
### Thanks for reading